# 1. Loading the fMRI images

Let's first learn how to load fMRI images, and also understand the dimensions of fMRI images.

In [1]:
#main library for preprocessing fMRI images
import nibabel as nb
print(nb.__version__)

3.2.1


In [2]:
# utilities
from pathlib import Path            #combine path elements with /

/Users/chaklam/DSAI/Environments/teaching_env/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


Setting the path directory

In [3]:
data_dir = Path('data')

## 1. Importing data

### Loading

Load some data.  Note that the data is from https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/29352  which contains 30 subjects data.  10 for AD (Alzheimers) (denoted nothing), 10 for MCI (Mild Cognitive Impairement) (denoted 1), and 10 for NCS (Normal) (denoted 2).  

Each of the subject has two information - functional and structual.

Let's try load the eighth (i just simply randomly choose) AD subject images.

In [4]:
t1w  = nb.load(data_dir /'subj_08_structural.nii')  #t1w stands for T1 weighted image
bold = nb.load(data_dir /'subj_08_functional.nii') #blood oxygenization image (the functional MRI)
t1w